# Prueba ETL

## Extract
Utilizando la API de Twitter https://developer.twitter.com/en/docs/twitter-api extraer datos de su preferencia (Tweets, Perfiles de usuario, Tendencias). Se deben realizar las peticiones necesarias para conformar un dataset de mínimo 100 filas.

#### Instalación de libreria para la API de Twitter

In [1]:
# pip install tweepy

#### Importamos librerias a utilizar

In [2]:
import tweepy
import pandas as pd
import datetime as dt
import requests
import json

#### Credenciales de autenticación de Twitter

In [3]:
api_key = 'kWVruxqw0omFMcHl345E5S3Xs'
api_key_secret = 'YLVoQrKeoDqUkXikuMIOLVOR2KUnDM7ex4n6XfQH9yiJt0lpLW'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAIIvlAEAAAAABaAEmEIWxav5okuUSa0f2f2S6c0%3DlyBMqaiM6T6s1Ybe0VGkTLoF0dCUv7tHi3JOmtEEZbd4ypQ3jI'
access_token = '1566302864930217984-prviE6p5uXez0NnI9dFedsu7qWbQ1a'
access_token_secret = '0cc9pyy3thSgVxM93MTKo9VQx3twpfJyq4d9HRUfWWOml'

#### Realizamos la autenticación con las credenciales

In [4]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

#### Creamos un objeto para realizas todas las llamadas a la API

In [5]:
api = tweepy.API(auth, wait_on_rate_limit = True)

#### Asignamos la cantidad de filas del dataset y obtenemos una linea de tiempo de los Tweets realizados por usuarios seguidos de un perfil

In [6]:
rows = 150
public_tweets = tweepy.Cursor(api.home_timeline).items(rows)
#public_tweets = api.home_timeline()
#for tweet in public_tweets:
#    print(f'{tweet.user.screen_name}:\n{tweet.text}\n{"*"*60}')

#### Creamos las columnas del dataset y lo rellenamos con la información consultada a la API

In [7]:
columns = ['Date and Time','User','Followers','Following','Total Tweets','Tweet','Language','Likes','Retweet','Verified account']
data = []

for tweet in public_tweets:
    data.append([tweet.created_at, tweet.user.screen_name, tweet.user.followers_count, tweet.user.friends_count,
                 tweet.user.statuses_count, tweet.text, tweet.lang, tweet.favorite_count, tweet.retweet_count,
                 tweet.user.verified])

Rate limit reached. Sleeping for: 3


## Transform
Realizar el proceso de transformación, limpieza y normalización que considere necesario sobre los datos utilizando la librería Pandas. Tener en cuenta la calidad de los datos (Integridad, Consistencia, Relevancia) para garantizar un análisis fiable.

#### Convertimos los datos extraidos de la API a un dataframe de python

In [8]:
data = pd.DataFrame(data, columns = columns)
data.head(5)

,Date and Time,User,Followers,Following,Total Tweets,Tweet,Language,Likes,Retweet,Verified account
0,2023-01-08 04:32:11+00:00,FranciaMarquezM,1078914,1026,10109,RT @PoloDemocratico: 🟡#PoloDemocrático | 🫶 Hoy...,es,0,30,True
1,2023-01-08 04:31:17+00:00,JP_GHIBLI,1663186,0,5649,熊本でのネコバスの乗り方ガイド🐾 https://t.co/S9NTzIiALs https...,ja,1356,139,True
2,2023-01-08 04:10:06+00:00,Geo_AW,69582,841,26615,I CANT STOP FUCKING SMILING https://t.co/AKPv...,en,57,0,False
3,2023-01-08 04:09:22+00:00,Geo_AW,69582,841,26615,THE GREATEST DUO IN ANIME/MANGA HISTORY https:...,en,1086,173,False
4,2023-01-08 04:08:44+00:00,PokeFansTaiwan,4819,348,53363,RT @livedoornews: 【今日は何の日？】1月8日は「平成が始まった日」\n\n...,ja,0,2707,False


#### Cantidad de filas y columnas que componen nuestro dataframe

In [9]:
data.shape

(150, 10)

#### Cantidad de datos nulos de nuestro dataframe

In [10]:
data.isna().sum()

Date and Time       0
User                0
Followers           0
Following           0
Total Tweets        0
Tweet               0
Language            0
Likes               0
Retweet             0
Verified account    0
dtype: int64

#### Nombre de las columnas, cantidad de registros no nulos y tipo de datos de cada columna del dataframe

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   Date and Time     150 non-null    datetime64[ns, UTC]
 1   User              150 non-null    object             
 2   Followers         150 non-null    int64              
 3   Following         150 non-null    int64              
 4   Total Tweets      150 non-null    int64              
 5   Tweet             150 non-null    object             
 6   Language          150 non-null    object             
 7   Likes             150 non-null    int64              
 8   Retweet           150 non-null    int64              
 9   Verified account  150 non-null    bool               
dtypes: bool(1), datetime64[ns, UTC](1), int64(5), object(3)
memory usage: 10.8+ KB


#### Proceso de transformación y normalización de los datos

#### De la columna 'Date and Time' extraemos la Fecha y Hora en columnas separadas

In [12]:
data['Date'] = data['Date and Time'].dt.strftime('%x')
data['Time'] = data['Date and Time'].dt.strftime('%X')

#### Eliminamos la columna 'Date and Time' ya que no nos servirá mas

In [13]:
data.drop(['Date and Time'], axis = 1, inplace = True)

#### Observamos los datos de la columna 'Verified account'

In [14]:
data['Verified account'].value_counts()

True     128
False     22
Name: Verified account, dtype: int64

#### Reemplazamos los valores de esta columna para una mayor comprensión

In [15]:
data.replace({'Verified account': {True: 'Yes', False: 'No'}}, inplace = True)

#### Observamos los datos de la columna 'Language'

In [16]:
data['Language'].value_counts()

es     85
en     35
ca     10
ja      9
und     2
tl      2
pt      2
zxx     2
cy      1
ro      1
ko      1
Name: Language, dtype: int64

#### Creamos un diccionario con el idioma abreviado y su forma larga

In [17]:
lang = {'af': 'afrikaans', 'sq': 'albanés', 'am': 'amhárico', 'ar': 'árabe', 'hy': 'armenio', 'az': 'azerbaiyano',
        'eu': 'vasco', 'be': 'bielorruso', 'bn': 'bengalí', 'bs': 'bosnio', 'bg': 'búlgaro', 'ca': 'catalán',
        'ceb' : 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chino (simplificado)', 'zh-tw': 'chino (tradicional)',
        'co': 'corso', 'hr' : 'croata', 'cs': 'checo', 'da': 'danés', 'nl': 'holandés', 'en': 'inglés',
        'eo': 'esperanto', 'et': ' estonio', 'tl': 'filipino', 'fi': 'finlandés', 'fr': 'francés', 'fy': 'frisón',
        'gl': 'gallego', 'ka': 'georgiano', 'de': 'alemán' , 'el': 'griego', 'gu': 'gujarati', 'ht': 'criollo haitiano',
        'ha': 'hausa', 'haw': 'hawaiano', 'iw': 'hebreo', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungaro',
        'is': 'islandés', 'ig': 'igbo', 'id': 'indonesio', 'ga ': 'irlandés', 'it': 'italiano', 'ja': 'japonés',
        'jw': 'javanés', 'kn': 'kannada', 'kk': 'kazajo', 'km': 'khmer', 'ko':'coreano', 'ku': 'kurdo (kurmanji)',
        'ky': 'kirguís', 'lo': 'lao', 'la': 'latín', 'lv': 'letón', 'lt' : 'lituano', 'lb': 'luxemburgués',
        'mk': 'macedonio', 'mg': 'malgache', 'ms': 'malayo', 'ml': 'malayalam', 'mt': ' maltés', 'mi': 'maorí',
        'mr': 'marathi', 'mn': 'mongol', 'my': 'myanmar (birmano)', 'ne': 'nepalí', 'no': 'noruego', 'ps': 'pashto',
        'fa': 'persa', 'pl': 'polaco', 'pt': 'portugués', 'pa': 'punjabi', 'ro': 'rumano ', 'ru': 'ruso', 'sm': 'samoano',
        'gd': 'gaélico escocés', 'sr': 'serbio', 'st': 'sesotho', 'sn': 'shona', 'sd': 'sindhi ', 'si': 'sinhala',
        'sk': 'eslovaco', 'sl': 'esloveno', 'so': 'somalí', 'es': 'español', 'su': 'sundanés', 'sw': 'swahili',
        'sv': 'sueco', 'tg': 'tayiko', 'ta': 'tamil', 'te': 'telugu', 'th': 'thai', 'tr ': 'turco', 'uk': 'ucraniano',
        'ur': 'urdu', 'uz': 'uzbeko', 'vi': 'vietnamita', 'cy': 'galés', 'xh': 'xhosa', 'yi': 'yiddish',
        'yo': 'yoruba', 'zu': 'zulu', 'fil': 'filipino', 'he': 'hebreo', 'und': 'indefinido', 'mul': 'multiples idiomas',
        'zxx': 'sin contenido linguistico'}

#### Convertimos los datos de la columna 'Language' a su forma no abreviada para una mayor comprensión

In [18]:
data.replace({'Language': lang}, inplace = True)

#### Observamos los datos de la columna 'Language' con los cambios aplicados

In [19]:
data['Language'].value_counts()

español                      85
inglés                       35
catalán                      10
japonés                       9
portugués                     2
indefinido                    2
filipino                      2
sin contenido linguistico     2
galés                         1
coreano                       1
rumano                        1
Name: Language, dtype: int64

## Load
Cargar o almacenar los datos en una base de datos relacional (Postgresql, SQL Server) o en archivos planos en formato .csv

#### Organizamos las columnas del dataset

In [20]:
data = data.reindex(columns = ['Date','Time','User','Followers','Following','Total Tweets','Tweet','Language',
                               'Likes','Retweet','Verified account'])
data

,Date,Time,User,Followers,Following,Total Tweets,Tweet,Language,Likes,Retweet,Verified account
0,01/08/23,04:32:11,FranciaMarquezM,1078914,1026,10109,RT @PoloDemocratico: 🟡#PoloDemocrático | 🫶 Hoy...,español,0,30,Yes
1,01/08/23,04:31:17,JP_GHIBLI,1663186,0,5649,熊本でのネコバスの乗り方ガイド🐾 https://t.co/S9NTzIiALs https...,japonés,1356,139,Yes
2,01/08/23,04:10:06,Geo_AW,69582,841,26615,I CANT STOP FUCKING SMILING https://t.co/AKPv...,inglés,57,0,No
3,01/08/23,04:09:22,Geo_AW,69582,841,26615,THE GREATEST DUO IN ANIME/MANGA HISTORY https:...,inglés,1086,173,No
4,01/08/23,04:08:44,PokeFansTaiwan,4819,348,53363,RT @livedoornews: 【今日は何の日？】1月8日は「平成が始まった日」\n\n...,japonés,0,2707,No
...,...,...,...,...,...,...,...,...,...,...,...
145,01/07/23,12:40:29,Geo_AW,69582,841,26615,Zoro and Sanji on tonight’s one piece episode ...,inglés,2739,326,No
146,01/07/23,12:36:27,FCBarcelona,45788341,111,129280,"""It will be a very difficult game""\n\n — ...",inglés,2127,122,Yes
147,01/07/23,12:34:03,FCBarcelona,45788341,111,129280,⚡️ 𝙇𝙄𝙑𝙀 𝙉𝙊𝙒 ⚡️\nBarça manager Xavi Hernández s...,español,1346,80,Yes
148,01/07/23,12:26:52,GustavoBolivar,1450169,24444,60319,RT @RevistaSemana: #GustavoBolívarSeDestapa | ...,español,0,147,Yes


#### Exportamos el dataset a un archivo plano en formato .csv

In [21]:
data.to_csv('data_twitter.csv', index = False)